In [19]:
import pandas as pd
import json
import re

In [18]:
data = json.load((open('data_wrangling_json/data_wrangling_json/data/world_bank_projects.json')))

sample_json_df = pd.read_json('data_wrangling_json/data_wrangling_json/data/world_bank_projects.json')

In [17]:
country = sample_json_df.groupby('countryname')
country_description = country.describe()
country_count = country_description['approvalfy']['count'].sort_values(ascending = False)
top_ten = country_count[:10].to_frame()
#top_ten

In [16]:
theme = sample_json_df
theme = theme.astype(str)
theme = theme.drop_duplicates()

In [15]:
code_dict = {'1': 'Economic management',
            '2': 'Public sector governance',
            '3': 'Rule of law',
            '4': 'Financial and private sector development',
            '5': 'Trade and integration',
            '6': 'Social protection and risk management',
            '7': 'Social dev/gender/inclusion',
            '8': 'Human development',
            '9': 'Urban development',
            '10': 'Rural development',
            '11': 'Environment and natural resources management'}

code_df = pd.DataFrame.from_dict(code_dict, orient='index', columns=['Theme'])

In [14]:
code_list = []
theme_list = []

for row in theme.itertuples():
    line = row.mjtheme_namecode
    digits = re.findall(r'\d+', line)
    code_list.append(digits)

for dig in code_list:
    for item in dig:
        theme_list.append(code_dict[item])
        
theme_counts = {i:theme_list.count(i) for i in theme_list}

listofTuples = sorted(theme_counts.items(), reverse=True, key=lambda x: x[1])

print('Top 10 Major World Bank Project Themes:\n')
for elem in listofTuples[:10]:
    print(elem[0], ":", elem[1] )

Top 10 Major World Bank Project Themes:

Environment and natural resources management : 250
Rural development : 216
Human development : 210
Public sector governance : 199
Social protection and risk management : 168
Financial and private sector development : 146
Social dev/gender/inclusion : 130
Trade and integration : 77
Urban development : 50
Economic management : 38


In [25]:
from pandas.io.json import json_normalize

df_themes = json_normalize(raw, 'mjtheme_namecode', ['id'])

df = pd.read_json('data_wrangling_json/data_wrangling_json/data/world_bank_projects.json')


with open('data_wrangling_json/data_wrangling_json/data/world_bank_projects.json') as f:
    raw = json.load(f)


df_themes_name_to_code = df_themes.groupby('name').code.max().sort_values(ascending=False)
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != '']
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code,columns=['code'])
df_themes_name_to_code['name_clean'] = df_themes_name_to_code.index
df_themes_code_to_name = df_themes_name_to_code.set_index(['code'])
df_themes_clean = df_themes.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)
df_theme_names = pd.DataFrame(df_themes_clean.groupby('id').apply(lambda x: '|'.join(x['name_clean'])),columns=['theme_names'])
df_theme_codes = pd.DataFrame(df_themes.groupby('id').apply(lambda x: '|'.join(x['code'])),columns=['theme_codes'])
df_theme_lut = df_theme_names.merge(df_theme_codes,left_index=True,right_index=True)
df_theme_lut.head()
df_cleaned = df.merge(df_theme_lut,how='right',left_on=['id'],right_index=True,)
df_cleaned[['id','mjtheme_namecode','theme_names','theme_codes']]

,id,mjtheme_namecode,theme_names,theme_codes
0,P129828,"[{'code': '8', 'name': 'Human development'}, {...",Human development|Environment and natural reso...,8|11
1,P144674,"[{'code': '1', 'name': 'Economic management'},...",Economic management|Social protection and risk...,1|6
2,P145310,"[{'code': '5', 'name': 'Trade and integration'...",Environment and natural resources management|S...,5|2|11|6
3,P144665,"[{'code': '7', 'name': 'Social dev/gender/incl...",Social dev/gender/inclusion|Social dev/gender/...,7|7
4,P144933,"[{'code': '5', 'name': 'Trade and integration'...",Trade and integration|Financial and private se...,5|4
5,P146161,"[{'code': '6', 'name': 'Social protection and ...",Social protection and risk management|Social p...,6|6
6,P121185,"[{'code': '2', 'name': 'Public sector governan...",Public sector governance|Financial and private...,2|4
7,P127033,"[{'code': '11', 'name': 'Environment and natur...",Human development|Environment and natural reso...,11|8
8,P130164,"[{'code': '10', 'name': 'Rural development'}, ...",Social dev/gender/inclusion|Rural development,10|7
9,P130903,"[{'code': '2', 'name': 'Public sector governan...",Public sector governance|Public sector governa...,2|2|2
